# Walkthrough on ChristmAIs API
Let's spread some algorithmic cheer throughout the holidays! This time, Thinking Machines Data Science presents you a package, `christmais`, to
generate abstract art using the idea of perception engines from [Tom White](https://medium.com/artists-and-machine-intelligence/perception-engines-8a46bc598d57). This notebook serves as a short walkthrough on how to use this API. 

In [ ]:
# Ensures that we're using the local version of christmAIs
import sys
sys.path.append('../')

# Some IPython magic to autoreload modules
%load_ext autoreload
%autoreload 2

In [ ]:
# Import modules
from christmais import (get_fasttext_pretrained, Artist)
from christmais import Predictor
from christmais import Trainer
import numpy as np

## FastText: Get a word embedding from text

The first step in this process is to get a word embedding from a given string input. For this we use gensim's FastText algorithm to do just that. The loaded `FastText` model was pretrained from the `brown` corpus. It turns out that for our purposes, training on this corpus is enough.

First, it checks if you have a trained `brown_fasttext.model` model in your `/tmp` directory. If there's none, then it will automatically train and save one for you!

In [ ]:
model = get_fasttext_pretrained(load=True)

In case you found the `brown` dataset insufficient and you wish to train on your own data, then worry not! We provided a `FastTextWrapper` so that you can work on your own data. Simply create an instance of the `FastTextWrapper` by providing your own sentences and additional keyword arguments as with gensim's [FastText](https://radimrehurek.com/gensim/models/fasttext.html#gensim.models.fasttext.FastText):

```python
from christmais.models import FastTextWrapper
#  Assuming you already have your own corpus loaded
model = FastTextWrapper(sentences=my_own_corpus, **kwargs)
```

Once we have our trained model, we can then create a word embedding out of any input string:

In [ ]:
embedding = model.transform("Thinking Machines Data Science")
print(embedding)

The embedding is represented as a vector of size 8, since this is the same configuration used in the original perception engine. For the next step, we will then use this embedding as a "seed" for our drawing system.

## Artist: Draw an image using the word embedding

In this step, we call our drawing system, aptly named `Artist`, to create an abstract image using the embedding as a seed. Here, we took inspiration from Tom White's [dopes repository](https://github.com/dribnet/dopes).

In [ ]:
artist = Artist(embedding, dims=(224, 224))

Once we have an artist, we can now generate some abstract art!

In [ ]:
artist.draw()

Note that each instatiation of the `Artist` class determines the color scheme of the resulting image. The positions of the lines and circles change whenever the `draw()` method is called. So if you want different palettes, instantiate multiple `Artist`s. Each of them can `draw()` different images whenever called.

### A note on genes

For our optimization routine, we exposed the positions of the circles and lines in an attribute called "gene." Each image has its own gene, and you can always create the same image given same genes. To get the gene value of the **last drawn image**, just call `get_gene()`

In [ ]:
gene = artist.get_gene()
print(gene)

As you can see, the `gene` is simply a matrix of shape `(density, 26)`. Again, we can re-draw the same image once we know its gene. We do that by calling `draw_from_gene()`

In [ ]:
artist.draw_from_gene(gene)

## Predictor: Predict the abstract art's nearest ImageNet class

Once we have an image, what we can do next is pass it to a classifier, in this case Resnet-152, to estimate how near it is to our target ImageNet class. The `Predictor` class gives us a handy way to abstract that. By supplying a target ImageNet class, we can call the `predict()` method and it will give us the confidence/score on how well it predicted our target class.

In [ ]:
p = Predictor()

In [ ]:
img = artist.draw_from_gene(gene) # We'll use the same image we got earlier
scores, results = p.predict(X=img, 
                            target="iron") # We'll see how well it predicts the class "iron"

In [ ]:
print('The score of predicting `iron` is {}, not that really high'.format(scores))

In addition, we can also check the nearest classes it predicted. Here, we can call the method `plot_results` while supplying the `results` dictionary from `predict`

In [ ]:
p.plot_results(results)

## Trainer: Optimize your images to look like the target class

As we've seen the score for our target class is rather low. What we will do next is optimize the images generated by our `Artist`s using an optimization algorithm. For `christmais`, we're using a genetic algorithm. Now, we will be introducing the `Trainer` class that integrates the three components (`FastText`, `Artist`, `Predictor`) together in a nice API.

In [ ]:
string = 'Thinking Machines Data Science' # string to get an embedding from
t = Trainer(string)

Given this, we can now call the `train()` method. It returns the best individual that we can call to draw

In [ ]:
best_gene = t.train(target='iron', steps=10)

There are a lot of options available inside `train`. I'd recommend that you call `help(t.train)` to see all available parameters. One thing that you might use is the `outdir` parameter. If supplied with a directory name, it draws the resulting image *per generation* in that directory. 

Now, let's examine `best_gene`:

In [ ]:
best_gene.artist.draw()

### Setting the colorscheme beforehand

We can play with the optimization parameters and set the colorschme beforehand. Here we can call `t.set_colors` and pass it a dictionary of colors with keys: `background`, `layer1`, `layer2`, `layer3`, and `lines`.

In [ ]:
string = 'Thinking Machines Data Science' # string to get an embedding from
t1 = Trainer(string) # let's make a new trainer

# Set colorscheme
new_colorscheme = {
    'background' : (255, 255, 255, 255),
    'layer1' : (255, 0, 0, 255),
    'layer2' : (0, 255, 0, 255),
    'layer3' : (0, 255, 255, 255),
    'lines'  : (0, 0, 0, 255)
}
t1.set_colors(new_colorscheme)

In [ ]:
best_gene1 = t1.train(target='iron', steps=10)

In [ ]:
best_gene1.artist.draw()